In [2]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
from torch.utils.data import Dataset
#from torchvision.datasets import MNIST
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
#import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn import preprocessing

#from torchvision.transforms import ToTensor
#from torchvision.utils import make_grid
from torch.utils.data import random_split
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from torch import Tensor
from torch.nn.parameter import Parameter
import torch.nn.init
from torch.nn.modules import Module
import math

from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
# Use a white background for matplotlib figures
matplotlib.rcParams['figure.facecolor'] = '#ffffff'

import warnings
warnings.filterwarnings("ignore")

In [30]:
def calculate_metrics(outputs, labels):    
    labels_onehot = torch.nn.functional.one_hot(labels, num_classes=2).cpu().detach().numpy()
    _, preds = torch.max(outputs, dim=1)
    #print(outputs.shape)
    outputs = outputs.cpu().detach().numpy()
    outputs = outputs.reshape(outputs.shape[0]*outputs.shape[1],outputs.shape[2])
    labels_onehot = labels_onehot.reshape(labels_onehot.shape[0]*labels_onehot.shape[1],labels_onehot.shape[2])
    #acc = torch.tensor(torch.sum(preds == labels).item() / len(preds)).mean()
    auc = np.round(roc_auc_score(labels_onehot,outputs),4)
    mcc=0
    f=0
    #mcc = matthews_corrcoef(labels,preds)
    #f = f1_score(labels,preds, average='weighted')
    return auc, mcc, f

In [31]:
#Define accuracy metric
def accuracy(outputs, labels):
    labels_onehot = torch.nn.functional.one_hot(labels, num_classes=10).cpu().detach().numpy()
    _, preds = torch.max(outputs, dim=1)
    acc = torch.tensor(np.round(torch.sum(preds == labels).item() / len(preds),4))
    #global_outputs.append(outputs.detach().numpy())
    return acc

In [32]:
class HyperNet(nn.Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight1: Tensor
    bias1: Tensor
    bias2: Tensor
    weight2: Tensor
    zval: Tensor
        
    def __init__(self, in_features: int, out_features: int, z_dim: int, bias: bool = False,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(HyperNet, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.z_dim = z_dim
        self.weight1 = nn.Parameter(torch.fmod(torch.randn((in_features,z_dim),**factory_kwargs),2))
        self.weight2 = nn.Parameter(torch.fmod(torch.randn((z_dim,out_features),**factory_kwargs),2))
        if bias:
            self.bias1 = nn.Parameter(torch.fmod(torch.randn((z_dim), **factory_kwargs),2))
            self.bias2 = nn.Parameter(torch.fmod(torch.randn((out_features), **factory_kwargs),2))
        else:
            self.register_parameter('bias', None)
        
    def forward(self, xb):  
        print("fwd")
        print(xb.shape)
        print(self.weight1.shape)
        print(self.bias1.shape)
        print(self.weight2.shape)
        print(self.bias2.shape)
        xb = torch.matmul(xb,self.weight1) + self.bias1
        # NO activation required
        # output of hypernet layer2
        xb = torch.matmul(xb,self.weight2) + self.bias2
        #return weights for main network
        return xb

In [33]:
#Define model
class MnistModel(nn.Module):
    def __init__(self, in_size, hidden_size, out_size, z_dim):
        super().__init__()
        # hidden layer
        self.linear1 = HyperNet(in_size, hidden_size, z_dim, bias=True)
        # output layer
        self.linear2 = HyperNet(hidden_size, out_size, z_dim, bias=True)
        
    def forward(self, xb):
        # Flatten the image tensors
        ##xb = (xb.view(xb.size(0), -1)).clone().detach()
        # Get intermediate outputs using hidden layer
        out = self.linear1(xb)
        # Apply activation function
        out = F.tanh(out)
        # Get predictions using output layer
        out = self.linear2(out)
        return out
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)
        # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'loss': loss, 'acc': acc, 'out': out, 'labels': labels}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        batch_out = [x['out'] for x in outputs]
        epoch_out = torch.stack(batch_out)
        batch_labels = [x['labels'] for x in outputs]
        epoch_labels = torch.stack(batch_labels)
        epoch_auc, mcc, f = calculate_metrics(epoch_out, epoch_labels)
        return {'epoch_loss': epoch_loss.item(), 'epoch_acc': epoch_acc.item(), 'epoch_auc': epoch_auc.item()}
    
    def epoch_end(self, epoch, result):
        return
        #print("Epoch [{}], loss: {:.4f}, acc: {:.4f}, auc score: {:.4f}".format(epoch, result['epoch_loss'],result['epoch_acc'],result['epoch_auc']))

In [34]:
#split training and validation indices
def split_indices(n, val_pct):
    n_val = int(val_pct*n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

In [35]:
#Move to cuda if available
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    else:
        return data.to(device, non_blocking=True)
    
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
            
    def __len__(self):
        return len(self.dl)

In [36]:
#Define fit function
def evaluate(model, loader):
    """Evaluate the model's performance on the validation set"""
    outputs=[]
    for batch in loader:
        outputs.append(model.validation_step(batch))
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, test_loader, opt_func=torch.optim.SGD):
    """Train the model using gradient descent"""
    history = []
    last_loss=0
    error_queue = []
    avg_error=1
    last_iter=0
    #loss_threshold=0.37
    row['Learning Rate'] = lr
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        last_iter = epoch + 1
        total_loss=0
        num_batches=0
        for batch in train_loader:
            loss = model.training_step(batch)
            
            total_loss = total_loss + loss
            num_batches = num_batches+1
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        avg_loss = total_loss/num_batches
        change = abs(last_loss - avg_loss)
        #if loss < loss_threshold:
        if len(error_queue) < 2:
            error_queue.append(abs(change))
            #print(error_queue)
        else:
            error_queue.append(abs(change))
            #print(error_queue)
            avg_error = sum(error_queue[1:2])/len(error_queue[1:2])
            error_queue.pop(0)

            #print(avg_error)       
            if avg_error <= 0.00000001: #Error threshold based early stopping
                break

        last_loss = loss
            
    row["Iter/Generation"] = last_iter
    # Training phase
    result = evaluate(model, train_loader)
    model.epoch_end(epochs, result)
    history.append(result.copy())
    
    # Validation phase
    result = evaluate(model, val_loader)
    model.epoch_end(epochs, result)
    history.append(result.copy())
    
    # Testing Phase
    result = evaluate(model, test_loader)
    model.epoch_end(epochs, result)
    history.append(result.copy())
    
    return history

In [37]:
class CustomDataset(Dataset):
    def __init__(self, annotations_file, transform=None, target_transform=None):
        df = pd.read_csv(annotations_file)
        self.labels = df['label']
        input_size = len(df.columns)-1
        self.data = df.iloc[:,0:input_size]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        x = self.data.values
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = np.round(min_max_scaler.fit_transform(x), 3)
        self.data = pd.DataFrame(x_scaled)
        
        label_read = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        data_read = torch.tensor(self.data.iloc[idx], dtype=torch.float32)
        return data_read, label_read

In [38]:
def get_data():
    batch_size=25
    '''Date Input/Output code for CSV dataset'''
    device = get_default_device()
    #df = pd.read_csv('../Dataset/pneumonia/pneumonia_train.csv')
    df = pd.read_csv('../Dataset/Wisconsin_Preprocessed.csv')
    #print(df.shape)
    
    num_classes=2
    # Reading the input data.
    dataset = torch.Tensor(np.array(df))
    input_size = len(df.columns)-1
    df = df.iloc[:,0:input_size]
    dataset = CustomDataset("../Dataset/Wisconsin_Preprocessed.csv")
    train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6,0.2,0.2], generator=torch.Generator())
    train_loader = DataLoader(train_dataset, batch_size,drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size,drop_last=True)
    test_loader = DataLoader(test_dataset, batch_size,drop_last=True)
    
    train_loader = DeviceDataLoader(train_loader, device)
    val_loader = DeviceDataLoader(val_loader, device)
    test_loader = DeviceDataLoader(test_loader, device)
    
    return train_loader, val_loader, test_loader

In [39]:
import os
def save_to_csv(df, name):
    path = 'Output/' + name
    if not os.path.exists('Output'):
        os.makedirs('Output')
    df.to_csv(path, index=False)

def create_csv():
    columns = ['Dataset', 'Hidden Layer', 'Output Layer',
               'Iter/Generation', 'Input Size', 
               'Learning Rate', 'Train Accuracy',
               'Train AUC', 'Val Accuracy',
               'Val AUC', 'Test Accuracy',
               'Test AUC', 'Time Taken(s)',
               'Time per generation', 'Memory Required', 'Total Memory(kB)',
               'Algorithmic Complexity']
    
    return pd.DataFrame(columns=columns)

In [ ]:
import time

def run(layer, train_loader, val_loader, test_loader):
    global df, input_size
    #print(hidden_size)
    z_dim=hidden_size/2
    start = time.time()
    #result = [{'epoch_acc': 1, 'epoch_auc': 2}, {'epoch_acc': 1, 'epoch_auc': 2}, {'epoch_acc': 1, 'epoch_auc': 2}]
    result = fit(500, 0.005, model, train_loader, val_loader, test_loader)
    row['Time Taken(s)'] = time.time() - start
    row['Time per generation'] = row['Time Taken(s)'] / row['Iter/Generation']
    row['Input Size'] = input_size
    #row['Num Classes'] = num_classes
    #row['Z Dim'] = z_dim
    
    row['Train Accuracy'] = result[0]['epoch_acc']
    row['Train AUC'] = result[0]['epoch_auc']
    row['Val Accuracy'] = result[1]['epoch_acc']
    row['Val AUC'] = result[1]['epoch_auc']
    row['Test Accuracy'] = result[2]['epoch_acc']
    row['Test AUC'] = result[2]['epoch_auc']
    #sum_hidden = np.array(layer).sum()
    memory_req = (input_size + hidden_size + num_classes) * z_dim
    row['Memory Required'] = memory_req
    row['Total Memory(kB)'] = memory_req/1024.0
    row['Algorithmic Complexity'] = (memory_req * row['Iter/Generation'])/10000.0
    row['Output Layer'] = num_classes
    row['Hidden Layer'] = layer

    df1 = pd.DataFrame(row, index=[0])
    df = pd.concat([df, df1], ignore_index=True)

df = create_csv()
if __name__ == '__main__':
    input_size = 0
    num_classes = 0
    #hidden_size = 10 # you can change this
    #layers = [10, 20, 50, 100, 200]
    layers = [10]
    #hidden_size = layers
    row = dict()
    
    for i in range(10):
        print(i)
        row['Dataset'] = i+1
        train_loader, val_loader, test_loader = get_data()
        for layer in layers:
            for batch in train_loader:
                image, labels = batch
                input_size = len(image.T)
                #print(input_size)
                break
            #input_size = len(train_loader.columns) - 1
            num_classes = 2
            hidden_size=layer
            model = MnistModel(input_size, hidden_size=layer, out_size=num_classes, z_dim=5)
            model = model.to(device=get_default_device())
            run(layer, train_loader, val_loader, test_loader)
        #end = time.time()
        #row['Time per generation'] = end - start
    
        display(df)
file_name = f'HN_metabric{time.time()}.csv'
save_to_csv(df, file_name)

0
fwd
torch.Size([25, 30])
torch.Size([30, 5])
torch.Size([5])
torch.Size([5, 10])
torch.Size([10])
fwd
torch.Size([25, 10])
torch.Size([10, 5])
torch.Size([5])
torch.Size([5, 2])
torch.Size([2])
fwd
torch.Size([25, 30])
torch.Size([30, 5])
torch.Size([5])
torch.Size([5, 10])
torch.Size([10])
fwd
torch.Size([25, 10])
torch.Size([10, 5])
torch.Size([5])
torch.Size([5, 2])
torch.Size([2])
fwd
torch.Size([25, 30])
torch.Size([30, 5])
torch.Size([5])
torch.Size([5, 10])
torch.Size([10])
fwd
torch.Size([25, 10])
torch.Size([10, 5])
torch.Size([5])
torch.Size([5, 2])
torch.Size([2])
fwd
torch.Size([25, 30])
torch.Size([30, 5])
torch.Size([5])
torch.Size([5, 10])
torch.Size([10])
fwd
torch.Size([25, 10])
torch.Size([10, 5])
torch.Size([5])
torch.Size([5, 2])
torch.Size([2])
fwd
torch.Size([25, 30])
torch.Size([30, 5])
torch.Size([5])
torch.Size([5, 10])
torch.Size([10])
fwd
torch.Size([25, 10])
torch.Size([10, 5])
torch.Size([5])
torch.Size([5, 2])
torch.Size([2])
fwd
torch.Size([25, 30])
tor

In [19]:
display(df)

,Dataset,Hidden Layer,Output Layer,Iter/Generation,Input Size,Learning Rate,Train Accuracy,Train AUC,Val Accuracy,Val AUC,Test Accuracy,Test AUC,Time Taken(s),Time per generation,Memory Required,Total Memory(kB),Algorithmic Complexity
